In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys,os, re
import pandas as pd
from functools import partial
import itertools
import json, glob

    
repobase="/Users/rosamartinezcorral/OneDrive - CRG - Centre de Regulacio Genomica/papers/github_repos/MPRAfitting/"
sys.path.append(os.path.join(repobase,"bin"))
from functions_fractional_occupancy import fi_1sites, fi_2sites, fi_3sites, fi_4sites, fi_5sites, fi_6sites
from biophysmodel_utils import logfoldchange_from_avbinding_ratesdirectly_Hill, get_S_max
from utils_fit import run_genetic, get_parsetnames, get_meanabserrorperaffinity_avbinding, error_parset_avbinding_celltypedifbyx
from utils_fit import dataframe_from_bestpars
evaluate_parset_function=get_meanabserrorperaffinity_avbinding
fifunctions={1:fi_1sites, 2:fi_2sites, 3:fi_3sites,4:fi_4sites,5:fi_5sites,6:fi_6sites}

In [2]:
#load the data
TF="Sp1"
data=pd.read_csv("./expdata/MPRA/SuppData.csv")
data
data

,ID,Sequence,Number of Sp1 binding sites,Affinity group,Mean binding site affinity,Sum of binding site affinities,Spacing between sites,Orientation of sites,Replicate 1,Replicate 2,Mean
0,Sp1.1,aggaccggatcaactgaatttgaaattacgttccaggcgtcggagt...,1,1.00,12.527789,12.527789,4,fwd,0.073337,-0.040363,0.018847
1,Sp1.2,aggaccggatcaactgctccgcggcttttggtaaaaataattcgac...,1,1.00,12.527789,12.527789,10,fwd,0.073805,0.147987,0.113257
2,Sp1.3,aggaccggatcaactcgacgaaggccggcaaatagttcatgggacc...,1,1.00,12.527789,12.527789,20,fwd,-0.102820,-0.169324,-0.133712
3,Sp1.4,aggaccggatcaactgtaagtgctgtcctacactgacagtgtgact...,1,1.00,12.527789,12.527789,4,rev,0.357273,0.246388,0.304191
4,Sp1.5,aggaccggatcaactgggcgctactgcgtcgatttaagtaatgaca...,1,1.00,12.527789,12.527789,10,rev,-0.294836,-0.286001,-0.288058
...,...,...,...,...,...,...,...,...,...,...,...
191,Sp1.192,aggaccggatcaactcagtcccggggaatgcatgaagttcgctccc...,6,0.25,6.965672,41.794034,10,tandem,-0.040295,-0.175021,-0.105298
192,Sp1.193,aggaccggatcaactttccctcatcgtagatcgactttaccgctgt...,6,0.25,6.857200,41.143202,20,tandem,-0.388067,-0.502576,-0.442961
193,Sp1.194,aggaccggatcaactgttacatacccgaatcgccagagtaatcatg...,6,0.10,4.673462,28.040772,4,tandem,0.358360,0.207757,0.285419
194,Sp1.195,aggaccggatcaactattcggcaagagtgccggttagaggaccatc...,6,0.10,4.784906,28.709437,10,tandem,0.239904,0.223515,0.234070


In [3]:
#load the Sp1 PWM, and define auxiliary function and variable needed to calculate S score
PWM_matrix=np.loadtxt(os.path.join(repobase,"PWMs","%s.csv"%TF),delimiter=",") #row: letter
index_dict={"A":0, "C":1, "G":2, "T":3} #first row of PWM: A, second: C, third: G, fourth: T
index_to_letter={val:key for key,val in index_dict.items()}
print(index_to_letter)
def get_reverse_comp(seq):
    comp={"A":"T", "C":"G", "G": "C", "T":"A"}
    compseq=[comp[s] for s in seq]
    return "".join(compseq[::-1])


{0: 'A', 1: 'C', 2: 'G', 3: 'T'}


In [4]:
sites_S=[] #add here a list with the S scores for each site in a sequence. If there are less than 6 sites, add NaN
for i in range(len(data)):
    Seq=data["Sequence"].values[i]
    orientation=data["Orientation of sites"].values[i]
    capital_substrings = re.findall(r'[A-Z]+', Seq)
    Svals=[]
    for j in range(6):
        if j<len(capital_substrings):
            if orientation=="fwd":
                site=capital_substrings[j]
            elif orientation=="rev":
                site=get_reverse_comp(capital_substrings[j])
            else:
                if j%2==0:
                    site=capital_substrings[j]
                else:
                    site=get_reverse_comp(capital_substrings[j])
    
            S=0
            for l,letter in enumerate(site):
                index=index_dict[letter]
                S+=np.log(PWM_matrix[index,l]/0.25)
            Svals.append(S)
        else:
            Svals.append(np.NaN)
    mean_S=np.nanmean(Svals)
    mean_S_table=data["Mean binding site affinity"].values[i]
    if not np.isnan(mean_S_table):
        if np.abs(mean_S-mean_S_table)<0.01: 
            print("similar S and average", mean_S_table, mean_S, Svals)
        else:
            print("!!!difference", mean_S_table, Svals)
    else:
        print("nan in table", mean_S, Svals)
    sites_S.append(Svals)
            
            
        

similar S and average 12.52778877 12.527788769365808 [12.527788769365808, nan, nan, nan, nan, nan]
similar S and average 12.52778877 12.527788769365808 [12.527788769365808, nan, nan, nan, nan, nan]
similar S and average 12.52778877 12.527788769365808 [12.527788769365808, nan, nan, nan, nan, nan]
similar S and average 12.52778877 12.527788769365808 [12.527788769365808, nan, nan, nan, nan, nan]
similar S and average 12.52778877 12.527788769365808 [12.527788769365808, nan, nan, nan, nan, nan]
similar S and average 12.52778877 12.527788769365808 [12.527788769365808, nan, nan, nan, nan, nan]
similar S and average 11.47606205 11.476062053360904 [11.476062053360904, nan, nan, nan, nan, nan]
similar S and average 11.47606205 11.476062053360904 [11.476062053360904, nan, nan, nan, nan, nan]
similar S and average 11.47606205 11.476062053360904 [11.476062053360904, nan, nan, nan, nan, nan]
nan in table 11.416087349513544 [11.416087349513544, nan, nan, nan, nan, nan]
similar S and average 11.416087

In [5]:
data.columns

Index(['ID', 'Sequence', 'Number of Sp1 binding sites', 'Affinity group',
       'Mean binding site affinity', 'Sum of binding site affinities',
       'Spacing between sites', 'Orientation of sites', 'Replicate 1',
       'Replicate 2', 'Mean'],
      dtype='object')

In [6]:
sites_S_df=pd.DataFrame(sites_S, columns=["affinity_site_%d"%j for j in range(1,7)])
data2=pd.concat([data, sites_S_df],axis=1)
data2.rename(columns={"Number of Sp1 binding sites": "nrepeats"},inplace=True)
data2

,ID,Sequence,nrepeats,Affinity group,Mean binding site affinity,Sum of binding site affinities,Spacing between sites,Orientation of sites,Replicate 1,Replicate 2,Mean,affinity_site_1,affinity_site_2,affinity_site_3,affinity_site_4,affinity_site_5,affinity_site_6
0,Sp1.1,aggaccggatcaactgaatttgaaattacgttccaggcgtcggagt...,1,1.00,12.527789,12.527789,4,fwd,0.073337,-0.040363,0.018847,12.527789,NaN,NaN,NaN,NaN,NaN
1,Sp1.2,aggaccggatcaactgctccgcggcttttggtaaaaataattcgac...,1,1.00,12.527789,12.527789,10,fwd,0.073805,0.147987,0.113257,12.527789,NaN,NaN,NaN,NaN,NaN
2,Sp1.3,aggaccggatcaactcgacgaaggccggcaaatagttcatgggacc...,1,1.00,12.527789,12.527789,20,fwd,-0.102820,-0.169324,-0.133712,12.527789,NaN,NaN,NaN,NaN,NaN
3,Sp1.4,aggaccggatcaactgtaagtgctgtcctacactgacagtgtgact...,1,1.00,12.527789,12.527789,4,rev,0.357273,0.246388,0.304191,12.527789,NaN,NaN,NaN,NaN,NaN
4,Sp1.5,aggaccggatcaactgggcgctactgcgtcgatttaagtaatgaca...,1,1.00,12.527789,12.527789,10,rev,-0.294836,-0.286001,-0.288058,12.527789,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Sp1.192,aggaccggatcaactcagtcccggggaatgcatgaagttcgctccc...,6,0.25,6.965672,41.794034,10,tandem,-0.040295,-0.175021,-0.105298,6.901010,7.058295,7.061587,6.937236,6.847482,6.988424
192,Sp1.193,aggaccggatcaactttccctcatcgtagatcgactttaccgctgt...,6,0.25,6.857200,41.143202,20,tandem,-0.388067,-0.502576,-0.442961,6.711255,7.058295,7.076221,6.746556,6.933641,6.617232
193,Sp1.194,aggaccggatcaactgttacatacccgaatcgccagagtaatcatg...,6,0.10,4.673462,28.040772,4,tandem,0.358360,0.207757,0.285419,4.413828,5.007064,4.582120,4.669917,4.549503,4.818339
194,Sp1.195,aggaccggatcaactattcggcaagagtgccggttagaggaccatc...,6,0.10,4.784906,28.709437,10,tandem,0.239904,0.223515,0.234070,4.984908,4.904738,4.426321,4.787191,5.072549,4.533729


In [24]:
data2.to_csv("./expdata/MPRA/SuppData_withAf.csv")